In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}

import org.apache.log4j._
Logger.getLogger("org").setLevel(Level.ERROR)

val spark = SparkSession.builder().getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://MSI:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1605303496590)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.log4j._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6b8c244b


In [2]:
val df = spark.read.format("libsvm").load("../../data/ml_scala/sample_linear_regression_data.txt")

df: org.apache.spark.sql.DataFrame = [label: double, features: vector]


In [3]:
df.show(5)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 5 rows



In [4]:
// Split data for train/test
val Array(train, test) = df.randomSplit(Array(0.8,0.2))

train: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]


In [7]:
// Create basic regression model
val lr = new LinearRegression()

// Create hyper-paramter grid for tuning
val paramGrid = (new ParamGridBuilder().addGrid(lr.regParam, Array(0.1,0.01))
                 .addGrid(lr.fitIntercept).addGrid(lr.elasticNetParam, Array(0.0,0.5,0.01))
                 .build())

// Validation for hyperparameter tuning - similiar to CV but only trains once
val trainValidationSplit = (new TrainValidationSplit().setEstimator(lr).setEvaluator(new RegressionEvaluator)
                            .setEstimatorParamMaps(paramGrid).setTrainRatio(0.8))

//  Train the model
val model = trainValidationSplit.fit(train)

// Show predictions against training set
model.transform(test).select("features", "label", "prediction").show()

org.apache.spark.SparkException:  Exception thrown in awaitResult:

In [6]:
spark.stop()